# MakerSpace Jam

In [1]:
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Primary Dependencies and Context Setting

In [2]:
!pip3 install -U -q openai==0.27.8 llama-index==0.8.6 nltk==3.8.1 python-dotenv

### Load the OPENAI API key

In [4]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]

### Context setting

In [5]:
from llama_index import ServiceContext
from llama_index.node_parser.simple import SimpleNodeParser
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index.llms import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding()
chunk_size = 1000
llm = OpenAI(
    temperature=0, 
    model="gpt-3.5-turbo",
    streaming=True
)

service_context = ServiceContext.from_defaults(
    llm=llm, 
    chunk_size=chunk_size,
    embed_model=embed_model
)

text_splitter = TokenTextSplitter(
    chunk_size=chunk_size
)

node_parser = SimpleNodeParser(
    text_splitter=text_splitter
)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


## Data Loading

### Wikipedia

In [6]:
!pip3 install -U -q wikipedia  

### Web

In [7]:
# list of wikipedia pages to index
webpages = [ "https://www.basketball-reference.com/" ]

##### Simple Web Reader

In [8]:
!pip3 install html2text

In [9]:
from llama_index import SimpleWebPageReader, TrafilaturaWebReader

web_docs = SimpleWebPageReader(html_to_text=True).load_data([webpages[0]])
web_docs[0].text

'  * [ Sports Reference ®](https://www.sports-reference.com/?utm_source=bbr&utm_medium=sr_xsite&utm_campaign=2023_01_srnav)\n  * [Baseball](https://www.baseball-reference.com/?utm_source=bbr&utm_medium=sr_xsite&utm_campaign=2023_01_srnav)\n  * [Football](https://www.pro-football-reference.com/?utm_source=bbr&utm_medium=sr_xsite&utm_campaign=2023_01_srnav) [(college)](https://www.sports-reference.com/cfb/)\n  * [Basketball](https://www.basketball-reference.com/?utm_source=bbr&utm_medium=sr_xsite&utm_campaign=2023_01_srnav) [(college)](https://www.sports-reference.com/cbb/)\n  * [Hockey](https://www.hockey-reference.com/?utm_source=bbr&utm_medium=sr_xsite&utm_campaign=2023_01_srnav)\n  * [Futebol](https://fbref.com/pt/?utm_source=bbr&utm_medium=sr_xsite&utm_campaign=2023_01_srnav)\n  * [Blog](https://www.sports-reference.com/blog/?utm_source=bbr&utm_medium=sr_xsite&utm_campaign=2023_01_srnav)\n  * [Stathead ®](https://stathead.com/?utm_source=bbr&utm_medium=sr_xsite&utm_campaign=2023_01_

##### Trafilatura Web Reader (seems better!!)

In [10]:
!pip3 install trafilatura

  Obtaining dependency information for trafilatura from https://files.pythonhosted.org/packages/96/47/d3523bdbf4e49233a17c2af6964b3dc9d8707c862847c49b324fbd7f9ba4/trafilatura-1.6.1-py3-none-any.whl.metadata
  Obtaining dependency information for courlan>=0.9.3 from https://files.pythonhosted.org/packages/29/f3/f63dc5fe0a474543b56c783ce4a11f33ca92b5e5c942dc8e57bd36e577f3/courlan-0.9.3-py3-none-any.whl.metadata
  Obtaining dependency information for htmldate>=1.4.3 from https://files.pythonhosted.org/packages/dc/6a/7730040fa8442722ebc9bc0f09c01a87f16d4de5ed8915626152bb1a2251/htmldate-1.5.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 25.3 MB/s eta 0:00:00
  Obtaining dependency information for lxml>=4.9.2 from https://files.pythonhosted.org/packages/ed/62/ffc30348ae141f69f9f23b65ba769db7ca209856c9a9b3406279e0ea24de/lxml-4.9.3-cp311-cp311-manylinux_2_28_x86_64.whl.metadata
  Using cached lxml-4.9.3-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.8

In [11]:
from llama_index import TrafilaturaWebReader

web_docs = TrafilaturaWebReader().load_data([webpages[0]])
web_docs[0].text

"Basketball Stats and History Statistics, scores, and history for the NBA, ABA, WNBA, and top European competition.\nEvery NBA & Every WNBA Player\nPlay Immaculate Grid\nPut your men's basketball knowledge to the test with our daily men's basketball trivia game. Can you complete the grid?\nRecent DebutsChance Comanche (POR), Jacob Gilyard (MEM), RaiQuan Gray (BRK), Justin Minaya (POR), Donovan Williams (ATL) and Jeenathan Williams (POR)\nNBA & ABA, WNBA, NBL, G League, and top International players\nIncludes indexed lists of players. International leagues include top European leagues and EuroLeague and EuroCup competitions, as well as China's CBA, Australia's NBL, and Men's Olympics.\nIn MemoriamHenry Dickerson, Charles Balentine, George Wilson, Henry Logan, Dedric Willoughby and Nikki McCray\nBorn On This DayMike Smrek, Mickey Johnson, Bruce Spraggins, Lionel Billingy, McCoy Ingram and James Robinson\nPlayer pages include basic statistics and links to player's game logs, splits, advan

## Setup Vector Store

This is intended to be a global vector store to insert the nodes from all data sources

In [12]:
!pip3 install -U -q chromadb==0.4.6 tiktoken==0.4.0 sentence-transformers==2.2.2 pydantic==1.10.11

In [13]:
from llama_index import VectorStoreIndex
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
import chromadb

In [14]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("all_data")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_index = VectorStoreIndex([], storage_context=storage_context, service_context=service_context)

INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.


### Construct Nodes and Insert-to-Store

Parse nodes for each loaded data source and insert it to the vector store.

In [15]:
# parse nodes from web docs and insert into vector index
for w_doc in web_docs:
    # print(wiki_doc)
    nodes = node_parser.get_nodes_from_documents([w_doc])
    for node in nodes:
        node.metadata = {'title': 'Basketball Stats and History'}
    vector_index.insert_nodes(nodes)

### Setup Metadata Filtering

First, we need to create our `VectoreStoreInfo` object which will hold all the relevant metadata we need for each component (in this case title metadata).

Notice that you need to include it in a text list.

In [16]:
from llama_index.tools import FunctionTool
from llama_index.vector_stores.types import (
    VectorStoreInfo,
    MetadataInfo,
    ExactMatchFilter,
    MetadataFilters,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine

from typing import List, Tuple, Any
from pydantic import BaseModel, Field

top_k = 3

vector_store_info = VectorStoreInfo(
    content_info="information about NBA players, teams, games, organizations, etc.",
    metadata_info=[MetadataInfo(
        name="title",
        type="str",
        description="info of player, team, organization, games, etc.",
    )]
)

# INSERTING METADATA WITH EXAMPLES

In [31]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

nba_wiki_web_docs = loader.load_data(pages=["Stephen_Curry", "Giannis_Antetokounmpo"], auto_suggest=False)

# parse nodes from web docs and insert into vector index
for w_doc in nba_wiki_web_docs:
    # print(w_doc)
    nodes = node_parser.get_nodes_from_documents([w_doc])
    for node in nodes:
        node.metadata = {'title': 'NBA Player Wiki Page'}
    vector_index.insert_nodes(nodes)

In [27]:
vector_index

In [39]:
from llama_index import Document
from llama_index.schema import MetadataMode

In [40]:
docPano = Document(
    text='NBA Player born Nov 1989 in Greece', 
    metadata={
        'type': 'player', 
        'number': '42', 
        'team': 'surfers', 
        'position': 'floating point guard',
        'specialty': 'eurostepping',
        'secret': 'loves pizza'
    }
)
docChristos = Document(
    text='NBA Player born Dec 1994 in the USA', 
    metadata={
        'type': 'player', 
        'number': '3', 
        'team': 'climbers', 
        'position': 'strong forward pass',
        'specialty': 'ally-oop',
        'secret': 'loves cats, candy and cartoons'
    },
    excluded_llm_metadata_keys=['secret'],
    metadata_seperator="::",
    metadata_template="{key}=>{value}",
    text_template="Metadata: {metadata_str}\n-----\nContent: {content}",
)

print("The LLM sees this: \n", docChristos.get_content(metadata_mode=MetadataMode.LLM))
print("The Embedding model sees this: \n", docChristos.get_content(metadata_mode=MetadataMode.EMBED))

The LLM sees this: 
 Metadata: type=>player::number=>3::team=>climbers::position=>strong forward pass::specialty=>ally-oop
-----
Content: NBA Player born Dec 1994 in the USA
The Embedding model sees this: 
 Metadata: type=>player::number=>3::team=>climbers::position=>strong forward pass::specialty=>ally-oop::secret=>loves cats, candy and cartoons
-----
Content: NBA Player born Dec 1994 in the USA


In [43]:
docSample = [docPano, docChristos]

In [ ]:
vector_index.insert_nodes(docSample)

In [52]:
filters = MetadataFilters(filters=[ExactMatchFilter(key="team", value="climbers")])
retriever = vector_index.as_retriever(filters=filters)
response = retriever.retrieve("when was player born?")
response[0]

TypeError: 'NodeWithScore' object is not subscriptable

In [57]:
query_engine = vector_index.as_query_engine(
    similarity_top_k=1,
    vector_store_query_mode="default",
    filters=MetadataFilters(
        filters=[
            ExactMatchFilter(key="team", value="climbers"),
        ]
    ),
    alpha=None,
    doc_ids=None,
)
response = query_engine.query("when was player born?")

In [63]:
response

Response(response='The player was born in December 1994.', source_nodes=[NodeWithScore(node=TextNode(id_='fbccdfaa-73fe-4992-9fa1-5f0a6b77c467', embedding=None, metadata={'type': 'player', 'number': '3', 'team': 'climbers', 'position': 'strong forward pass', 'specialty': 'ally-oop', 'secret': 'loves cats, candy and cartoons'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=['secret'], relationships={}, hash='7d8f5c058a3fe84d990210e51109f91470f01cea7405ccac2e137460e9e0dc63', text='NBA Player born Dec 1994 in the USA', start_char_idx=None, end_char_idx=None, text_template='Metadata: {metadata_str}\n-----\nContent: {content}', metadata_template='{key}=>{value}', metadata_seperator='::'), score=0.317190031781011)], metadata={'fbccdfaa-73fe-4992-9fa1-5f0a6b77c467': {'type': 'player', 'number': '3', 'team': 'climbers', 'position': 'strong forward pass', 'specialty': 'ally-oop', 'secret': 'loves cats, candy and cartoons'}})

In [64]:
response.metadata

{'fbccdfaa-73fe-4992-9fa1-5f0a6b77c467': {'type': 'player',
  'number': '3',
  'team': 'climbers',
  'position': 'strong forward pass',
  'specialty': 'ally-oop',
  'secret': 'loves cats, candy and cartoons'}}

---

Now we'll create our base PyDantic object that we can use to ensure compatability with our application layer. This verifies that the response from the OpenAI endpoint conforms to this schema.

In [69]:
class AutoRetrieveModel(BaseModel):
    query: str = Field(..., description="natural language query string")
    filter_key_list: List[str] = Field(
        ..., description="List of metadata filter field names"
    )
    filter_value_list: List[str] = Field(
        ...,
        description=(
            "List of metadata filter field values (corresponding to names specified in filter_key_list)"
        )
    )

Now we can build our function that we will use to query the functional endpoint.

>The `docstring` is important to the functionality of the application.

In [71]:
def auto_retrieve_fn(
    query: str, filter_key_list: List[str], filter_value_list: List[str]
):
    """Auto retrieval function.

    Performs auto-retrieval from a vector database, and then applies a set of filters.

    """
    query = query or "Query"

    exact_match_filters = [
        ExactMatchFilter(key=k, value=v)
        for k, v in zip(filter_key_list, filter_value_list)
    ]
    retriever = VectorIndexRetriever(
        vector_index, filters=MetadataFilters(filters=exact_match_filters), top_k=top_k
    )
    query_engine = RetrieverQueryEngine.from_args(retriever)

    response = query_engine.query(query)
    return str(response)

Now we need to wrap our system in a tool in order to integrate it into the larger application.

Source Code Here:
- [`FunctionTool`](https://github.com/jerryjliu/llama_index/blob/d24767b0812ac56104497d8f59095eccbe9f2b08/llama_index/tools/function_tool.py#L21)

In [72]:
description = f"""\
Use this tool to look up semantic information about NBA.
The vector database schema is given below:
{vector_store_info.json()}
"""

auto_retrieve_tool = FunctionTool.from_defaults(
    fn=auto_retrieve_fn, 
    name='auto_retrieve_tool',
    description=description, 
    fn_schema=AutoRetrieveModel
)

All that's left to do is attach the tool to an OpenAIAgent and let it rip!

Source Code Here:
- [`OpenAIAgent`](https://github.com/jerryjliu/llama_index/blob/d24767b0812ac56104497d8f59095eccbe9f2b08/llama_index/agent/openai_agent.py#L361)

In [73]:
from llama_index.agent import OpenAIAgent

agent = OpenAIAgent.from_tools(
    [auto_retrieve_tool], llm=llm, verbose=True
)

In [82]:
response = agent.chat("What are the current 'Trending Player' in the NBA?")
print(str(response))

=== Calling Function ===
Calling function: auto_retrieve_tool with args: {
  "query": "current Trending Player NBA",
  "filter_key_list": ["title"],
  "filter_value_list": ["Trending Player"]
}
Got output: None
I'm sorry, but I couldn't find any current information on 'Trending Player' in the NBA. Is there anything else I can assist you with?
